<a href="https://colab.research.google.com/github/amanullahshah32/Deep-Learning/blob/main/Kaggle_Competition/Lumbar_spine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install kaggle


In [8]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amanullahshah","key":"874de6bec3f185a0d03615b63bbc63d0"}'}

In [9]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle competitions download -c rsna-2024-lumbar-spine-degenerative-classification


 19% 5.43G/28.2G [04:25<23:16, 17.5MB/s]

In [ ]:
!unzip rsna-2024-lumbar-spine-degenerative-classification.zip -d /content/lumbar_spine_data


In [6]:
import os

In [7]:
dataset_dir = '/content/lumbar_spine_data/'
os.listdir(dataset_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/content/lumbar_spine_data/'

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Load and Preprocess the data with PyTorch

In [ ]:
!pip install pydicom


In [ ]:
import os
import pydicom
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class LumbarSpineDataset(Dataset):
    def __init__(self, images_dir, labels_csv, transform=None):
        self.images_dir = images_dir
        self.labels_df = pd.read_csv(labels_csv)
        self.transform = transform

        # Create a dictionary mapping the image IDs to their respective file paths
        self.image_paths = {}
        for root, dirs, files in os.walk(images_dir):
            for file in files:
                if file.endswith('.dcm'):
                    image_id = file.split('.')[0]  # Assuming the ID is the filename without extension
                    self.image_paths[image_id] = os.path.join(root, file)

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # Get the image ID and its corresponding file path
        img_id = str(self.labels_df.iloc[idx, 0])
        img_path = self.image_paths[img_id]

        # Load the DICOM image
        dicom_image = pydicom.dcmread(img_path).pixel_array
        dicom_image = dicom_image / dicom_image.max()  # Normalize the image

        # Convert to a 3-channel image if needed
        dicom_image = np.stack([dicom_image] * 3, axis=-1)
        dicom_image = dicom_image.astype(np.float32)

        if self.transform:
            dicom_image = self.transform(dicom_image)

        # Assuming labels are in the subsequent columns
        label = torch.tensor(self.labels_df.iloc[idx, 1:].values, dtype=torch.float32)

        return dicom_image, label



# Define the transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and dataloaders
train_images_dir = os.path.join(dataset_dir, 'train_images')
train_labels_csv = os.path.join(dataset_dir, 'train.csv')

train_dataset = LumbarSpineDataset(images_dir=train_images_dir, labels_csv=train_labels_csv, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# For validation, you can either split your training set or use a validation set if provided
# For example, splitting the train_dataset into train and validation


In [ ]:
from torch.utils.data import random_split

# Assume 80% training and 20% validation split
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create validation DataLoader
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


## Build and Train a Model in PyTorch

### Setup  device agnostic code

In [ ]:
# Set device to GPU if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Building a  model
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define the model
class SimpleCNN(nn.Module):
  def __init__(self):
      super(SimpleCNN, self).__init__()
      self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
      self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
      self.fc1 = nn.Linear(64 * 56 * 56, 128)
      self.fc2 = nn.Linear(128, 3)  # Adjust based on the number of classes

  def forward(self, x):
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 64 * 56 * 56)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x


# Instantiate and move the model to the device
model = SimpleCNN().to(device)
model

### Loss function and optimizer

In [ ]:

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Update the Training Loop

In [ ]:
# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the correct device

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the correct device
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')
